# CleanLab Task

In [ ]:
!pip install "cleanlab[datalab]"

In [ ]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from cleanlab.classification import CleanLearning

In [ ]:
# Load the Iris dataset
iris = load_iris()
df = pd.DataFrame(iris.data, columns=iris.feature_names)
df['target'] = iris.target
print(df.head())

In [ ]:
# Introduce anomalies by altering some values
np.random.seed(42)
anomaly_indices = np.random.choice(df.index, size=10, replace=False)
df.loc[anomaly_indices, 'petal length (cm)'] = np.random.uniform(5, 7, size=10)

In [ ]:
X = iris.data
y = iris.target

# Use CleanLearning for anomaly detection
clf = CleanLearning()
clf.fit(X, y)

# Find potential anomalies in labels
label_issues = clf.find_label_issues(X, y)

# Output the anomalies
anomalies = np.where(label_issues["is_label_issue"])[0]
print(f"Anomalies detected at indices: {anomalies}")
print(f"Suspected anomaly values: {X[anomalies]}")

In [ ]:
# Create an empty list to store DataFrames
suspect_dfs = []

flower_species = {0.0: "Setosa", 1.0: "Versicolor", 2.0: "Virginica"}

# Loop over the indices and create a structured DataFrame for each
for idx in anomalies:
    # Create a DataFrame for the suspected anomaly data point
    df_suspect = pd.DataFrame([df.iloc[idx][iris.feature_names].values], columns=iris.feature_names)
    df_suspect.insert(0, "Index", idx)  # Insert index column

    df_suspect["True Label"] = df.iloc[idx]["target"]
    df_suspect["Flower Species"] = flower_species[y[idx]]  # Map label to flower species

    # Append the current suspect DataFrame to the list
    suspect_dfs.append(df_suspect)

# Combine all the suspect DataFrames into a single DataFrame
df_all_suspects = pd.concat(suspect_dfs, ignore_index=True)

# Print the full table of suspected anomalies
print("\n                                      Suspected Anomalous Data Points")
print("-----------------------------------------------------------------------------------------------------------")

print(df_all_suspects.to_string(index=False))